## IPW with Regression Adjustment

In [1]:
import numpy as np
import pandas as pd
import scipy
from aipyw import IpwRa

In [2]:
def binary_dgp(n, k, pscore_fn, tau_fn, outcome_fn, k_cat=1):
    Sigma = np.random.uniform(-1, 1, (k, k))
    Sigma = Sigma @ Sigma.T
    Xnum = np.random.multivariate_normal(np.zeros(k), Sigma, n)
    # generate categorical variables
    Xcat = np.random.binomial(1, 0.5, (n, k_cat))
    X = np.c_[Xnum, Xcat]
    W = np.random.binomial(1, pscore_fn(X), n).astype(int)
    Y = outcome_fn(X, W, tau_fn)
    df = pd.DataFrame(
        np.c_[Y, W, X], columns=["Y", "W"] + [f"X{i}" for i in range(k + 1)]
    )
    return df

In [3]:
def outcome_fn(x, w, taufn):
    base_term = x[:, 0] + 2 * x[:, 1] + x[:, 3]
    return taufn(x) * w + base_term + np.random.normal(0, 1, len(w))


def pscore_fn(x):
    lin = x[:, 0] - x[:, 1] - x[:, 2] + x[:, 3] + np.random.normal(0, 1, x.shape[0])
    return scipy.special.expit(lin)


df = binary_dgp(n = 10_000, k = 4, tau_fn = lambda x: 1,
    pscore_fn = pscore_fn, outcome_fn = outcome_fn)
df.head()

,Y,W,X0,X1,X2,X3,X4
0,1.302749,0.0,-0.874835,0.644566,-0.801552,-0.656742,1.0
1,1.936335,0.0,0.801376,0.974530,-0.767204,0.564353,0.0
2,5.655286,1.0,0.483897,1.063274,-1.155463,0.630855,1.0
3,-3.287480,0.0,-0.266067,-2.103778,1.650526,0.614969,1.0
4,-0.930136,1.0,0.395542,0.242585,1.072401,-1.390246,1.0


## estimators

#### Regression adjustment

In [4]:
m = IpwRa(typ='ra')
m.fit(df)
m.summary()

{'est': 1.037, '95.0% CI (Bootstrap)': array([0.9893, 1.0783])}

#### IPW

In [5]:
m = IpwRa(typ='ipw')
m.fit(df)
m.summary()

{'est': 1.0103, '95.0% CI (Bootstrap)': array([0.9602, 1.0747])}

#### IPW

In [6]:
m = IpwRa(typ='ipwra')
m.fit(df)
m.summary()

{'est': 1.0097, '95.0% CI (Bootstrap)': array([0.962 , 1.0526])}